# Meteorology

```{warning} This is not meant to be a standalone notebook.
This notebook is part of the process we have for adding entries to the NCL Index and is not meant to be used as tutorial or example code.
```

## Functions covered
- [dewtemp_trh](https://www.ncl.ucar.edu/Document/Functions/Built-in/dewtemp_trh.shtml)
- [daylight_fao56](https://www.ncl.ucar.edu/Document/Functions/Crop/daylight_fao56.shtml)
- [satvpr_temp_fao56](https://www.ncl.ucar.edu/Document/Functions/Crop/satvpr_temp_fao56.shtml)
- [satvpr_tdew_fao56](https://www.ncl.ucar.edu/Document/Functions/Crop/satvpr_tdew_fao56.shtml)

## NCL code

```{literalinclude} ../ncl_raw/meteorology.ncl

```

## Python Functionality

### dewtemp_trh

In [ ]:
#### Collect NCL values for dewtemp_trh from geocat-datafiles
import geocat.datafiles as gdf
import numpy as np

dewtemp_data = gdf.get('applications_files/ncl_outputs/dewtemp_trh_output.txt')
dewtemp_data = np.loadtxt(dewtemp_data, delimiter=',', skiprows=6)

In [ ]:
### Collect NCL `dewtemp` value and associated (temperature_kelvin, relative humidity) values
ncl_dewtemp = {}
tk_rh = tuple(map(tuple, dewtemp_data[::, 0:2]))
dewtemp_values = dewtemp_data[::, 2]
ncl_dewtemp = dict(zip(tk_rh, dewtemp_values))

In [ ]:
### Collect Temperature (Kelvin) and Relative Humidity Pairs
tk_rh = []
for tk in range(273, 374 + 1):
    for rh in range(1, 100 + 1):
        tk_rh.append((tk, rh))

### Calculate GeoCAT-Comp `dewtemp` value and tk/rh
from geocat.comp import dewtemp

geocat_dewtemp = {}

for i, pair in enumerate(tk_rh):
    tk, rh = pair
    geocat_dewtemp[pair] = dewtemp(tk, rh) - 273.15

### daylight_fao56

In [ ]:
#### Collect NCL values for daylight_fao56 from geocat-datafiles
import geocat.datafiles as gdf
import numpy as np

daylight_data = gdf.get('applications_files/ncl_outputs/daylight_fao56_output.txt')
daylight_data = np.loadtxt(daylight_data, delimiter=',', skiprows=6)

In [ ]:
### Collect NCL `daylight_fao56` value and associated (doy, latitude) values
ncl_daylight = {}
doy_lat = tuple(map(tuple, daylight_data[::, 0:2]))
daylight_values = daylight_data[::, 2]
ncl_daylight = dict(zip(doy_lat, daylight_values))

In [ ]:
### Collect DOY and Latitude Pairs
doy_lat = []
for doy in range(0, 365 + 1):
    for lat in range(-66, 66 + 1):
        doy_lat.append((doy, lat))

### Calculate GeoCAT-Comp `daylight_fao56` value and doy/lat
from geocat.comp import max_daylight

geocat_daylight = {}

for i, pair in enumerate(doy_lat):
    doy, lat = pair
    geocat_daylight[pair] = max_daylight(doy, lat)

### satvpr_temp_fao56

In [ ]:
#### Collect NCL values for satvpr_temp_fao56 from geocat-datafiles
import geocat.datafiles as gdf
import numpy as np

satvpr_temp_fao56_data = gdf.get(
    'applications_files/ncl_outputs/satvpr_temp_fao56_output.txt'
)
satvpr_temp_fao56_data = np.loadtxt(satvpr_temp_fao56_data, delimiter=',', skiprows=6)

In [ ]:
### Collect NCL `satvpr_temp_fao56` value and associated (temp, satvpr_temp) values
ncl_satvpr_temp_fao56 = dict(
    zip(satvpr_temp_fao56_data[::, 0], satvpr_temp_fao56_data[::, 1])
)

In [ ]:
### Calculate GeoCAT-Comp `saturation_vapor_pressure`
from geocat.comp import saturation_vapor_pressure

geocat_satvpr_temp_fao56 = {}

for temp in range(33, 212 + 1):
    geocat_satvpr_temp_fao56[temp] = saturation_vapor_pressure(temp)

### satvpr_tdew_fao56

In [ ]:
#### Collect NCL values for satvpr_tdew_fao56 from geocat-datafiles
import geocat.datafiles as gdf
import numpy as np

satvpr_tdew_fao56_data = gdf.get(
    'applications_files/ncl_outputs/satvpr_tdew_fao56_output.txt'
)
satvpr_tdew_fao56_data = np.loadtxt(satvpr_tdew_fao56_data, delimiter=',', skiprows=6)

In [ ]:
### Collect NCL `satvpr_tdew_fao56` value and associated (temp, act_sat_vapr_pressure) values
ncl_satvpr_tdew_fao56 = dict(
    zip(satvpr_tdew_fao56_data[::, 0], satvpr_tdew_fao56_data[::, 1])
)

In [ ]:
### Calculate GeoCAT-Comp `actual_saturation_vapor_pressure`
from geocat.comp import actual_saturation_vapor_pressure

geocat_satvpr_tdew_fao56 = {}

for temp in range(33, 212 + 1):
    geocat_satvpr_tdew_fao56[temp] = actual_saturation_vapor_pressure(temp)

## Comparison

### dewtemp_trh

In [ ]:
import math

for pair in ncl_dewtemp.keys():
    try:
        assert math.isclose(
            ncl_dewtemp[pair], geocat_dewtemp[pair], rel_tol=1e-04
        )  # within 4 decimal points
    except Exception:
        assert math.isclose(
            ncl_dewtemp[pair], geocat_dewtemp[pair], rel_tol=1e-02
        )  # within 2 decimal points
        print(f"{pair}:")
        print(f"\t{ncl_dewtemp[pair]}, {geocat_dewtemp[pair]}")
        print(f"\tDifference: {ncl_dewtemp[pair] - geocat_dewtemp[pair]}")

### daylight_fao56

In [ ]:
import math

for pair in ncl_daylight.keys():
    assert math.isclose(
        ncl_daylight[pair], geocat_daylight[pair].flatten()[0], rel_tol=1e-05
    )  # within 5 decimal points

### satvpr_temp_fao56

In [ ]:
import math

for key in ncl_satvpr_temp_fao56.keys():
    assert math.isclose(
        ncl_satvpr_temp_fao56[key], geocat_satvpr_temp_fao56[key], rel_tol=1e-05
    )  # within 5 decimal points

### satvpr_tdew_fao56

In [ ]:
import math

for key in ncl_satvpr_tdew_fao56.keys():
    assert math.isclose(
        ncl_satvpr_tdew_fao56[key], geocat_satvpr_tdew_fao56[key], rel_tol=1e-05
    )  # within 5 decimal points